## Working with neural networks with Keras and Python

(based on the example by Sebastiaan Mathot and an experiment of Chris Longmore)

### Example 1: Train a neural network to correctly classify whether an observation is closer to 1 or to 0.

Input layer with 1 neuron - I  
Hidden layer with 8 neurons - H  
Output layer with 2 neurons - O  

I: O - O  

H: O - O - O - O - O - O - O - O  

O: O - O  

In [31]:
# Import modules
import numpy as np
from keras import Sequential
from keras.layers import Dense

#### Create training data and labels

In [32]:
data = np.random.random(10000)
data.shape = 10000, 1
labels = np.array(data >= .5, dtype=int)

#### Create the model

Sequential model with:  
1 Input  
1 Hidden layer with 8 neurons (dense with softmax - sigmoid styled output layer - activation)  
1 Output layer with 2 outputs  

In [33]:
model = Sequential(
    [
        Dense(units=8, input_shape=(1,), activation='relu'),
        Dense(units=2, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 8)                 16        
                                                                 
 dense_5 (Dense)             (None, 2)                 18        
                                                                 
Total params: 34
Trainable params: 34
Non-trainable params: 0
_________________________________________________________________


Summary shows:
- hidden layer with 8 units and 16 parameters (1x8 parameters for input connection and 1x8 parameters for bias)
- output layer with 2 units and 18 parameters (each of the 2 units has a bias and is connected to each of the 8 units in the hidden layer, i.e. 2+2*8)

#### Compile the model

1. Define a loss function to determine error:
   - sparse: expect the first output neuron to be most active and the second least active for 0 and vice versa for 1
2. Define optimizer that changes (reduces) the weights and bias parameters of the model during learning

In [34]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

#### Train model

Epochs: number of times the data is fed into the model ()

In [35]:
model.fit(
    x=data,
    y=labels,
    epochs=10,
    verbose=2,
)

Epoch 1/10
313/313 - 0s - loss: 0.6283 - accuracy: 0.7641 - 232ms/epoch - 742us/step
Epoch 2/10
313/313 - 0s - loss: 0.4571 - accuracy: 0.9290 - 104ms/epoch - 333us/step
Epoch 3/10
313/313 - 0s - loss: 0.2936 - accuracy: 0.9776 - 105ms/epoch - 334us/step
Epoch 4/10
313/313 - 0s - loss: 0.2053 - accuracy: 0.9873 - 108ms/epoch - 344us/step
Epoch 5/10
313/313 - 0s - loss: 0.1585 - accuracy: 0.9925 - 108ms/epoch - 345us/step
Epoch 6/10
313/313 - 0s - loss: 0.1304 - accuracy: 0.9945 - 109ms/epoch - 347us/step
Epoch 7/10
313/313 - 0s - loss: 0.1115 - accuracy: 0.9958 - 111ms/epoch - 353us/step
Epoch 8/10
313/313 - 0s - loss: 0.0978 - accuracy: 0.9975 - 120ms/epoch - 382us/step
Epoch 9/10
313/313 - 0s - loss: 0.0875 - accuracy: 0.9969 - 118ms/epoch - 376us/step
Epoch 10/10
313/313 - 0s - loss: 0.0795 - accuracy: 0.9981 - 120ms/epoch - 384us/step


#### Test the model on independent data

In [36]:
test_data = np.array([0, 0.2, 0.4, 0.6, 0.8, 1])
predictions = model.predict(test_data)
predictions

array([[9.99331892e-01, 6.68073015e-04],
       [9.97054935e-01, 2.94501288e-03],
       [8.93979371e-01, 1.06020644e-01],
       [1.01804771e-01, 8.98195207e-01],
       [1.52123626e-03, 9.98478830e-01],
       [8.53710808e-05, 9.99914646e-01]], dtype=float32)

In [37]:
# Check actual classifications to see whether the model is correctly classifying the test_data
print(np.argmax(predictions, axis=1))

[0 0 0 1 1 1]


#### Validate the data during the training

We actually rather want to keep a part of the original data separate and use it later to test the model in one go.

In [38]:
model.fit(
    x=data,
    y=labels,
    epochs=10,
    verbose=2,
    validation_split=0.1,
)

Epoch 1/10
282/282 - 0s - loss: 0.0742 - accuracy: 0.9972 - val_loss: 0.0639 - val_accuracy: 0.9990 - 177ms/epoch - 626us/step
Epoch 2/10
282/282 - 0s - loss: 0.0694 - accuracy: 0.9980 - val_loss: 0.0585 - val_accuracy: 1.0000 - 129ms/epoch - 458us/step
Epoch 3/10
282/282 - 0s - loss: 0.0652 - accuracy: 0.9969 - val_loss: 0.0549 - val_accuracy: 0.9990 - 128ms/epoch - 453us/step
Epoch 4/10
282/282 - 0s - loss: 0.0617 - accuracy: 0.9974 - val_loss: 0.0520 - val_accuracy: 0.9980 - 128ms/epoch - 454us/step
Epoch 5/10
282/282 - 0s - loss: 0.0584 - accuracy: 0.9980 - val_loss: 0.0477 - val_accuracy: 0.9980 - 133ms/epoch - 472us/step
Epoch 6/10
282/282 - 0s - loss: 0.0556 - accuracy: 0.9980 - val_loss: 0.0457 - val_accuracy: 0.9990 - 131ms/epoch - 465us/step
Epoch 7/10
282/282 - 0s - loss: 0.0531 - accuracy: 0.9979 - val_loss: 0.0431 - val_accuracy: 0.9990 - 130ms/epoch - 460us/step
Epoch 8/10
282/282 - 0s - loss: 0.0509 - accuracy: 0.9974 - val_loss: 0.0407 - val_accuracy: 0.9990 - 131ms/epo

### Example 2: Classify an image using a pretrained deep neural network

Training your own model to perform well is difficult and requires a lot of data. Luckily there is already pretty decent models readily available that are capable of classifying pictures.  
In this example, we will make use of such a pre-trained model, specifically the MobileNetV2.

In [39]:
# Import modules
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import numpy as np
from imageio import imread # library to read images as numpy array

In [40]:
model = MobileNetV2(weights="imagenet")
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

The summary shows MobileNetV2 is a deep but not very wide network (i.e. it has many layers with rather few neurons).

#### Load image to classify

We need to import the image and make keras understand it before we can classify it.  
We will first load it in and then preprocess it. This step is necessary because the loaded image is likely not to match input the model was trained on. Results would be weird.

In [41]:
data = np.empty((1, 224, 224, 3)) # (image_count, px height, px width, color channels)
data[0] = imread("data/images/elephant.jpg")
data = preprocess_input(data) # preprocess the image to get image values between -1 and 1
np.shape(data)

(1, 224, 224, 3)

#### Classify the image

In [42]:
predictions = model.predict(data)
predictions.shape, predictions # output of the 1000 neurons of the model

((1, 1000),
 array([[1.49515879e-04, 5.75295489e-05, 1.29174559e-05, 6.07296533e-05,
         9.21287283e-05, 1.23287391e-04, 7.33565030e-05, 3.37578786e-05,
         9.92799778e-06, 4.99068046e-05, 3.34968026e-05, 2.66517600e-05,
         7.22230034e-05, 2.58851851e-05, 5.17975095e-05, 6.53651950e-05,
         4.16641560e-05, 2.60621437e-05, 5.64925795e-05, 3.74715528e-05,
         4.78477450e-05, 5.64909060e-05, 1.39919575e-04, 3.62852297e-05,
         5.76817802e-05, 7.39133029e-05, 9.60881298e-05, 4.51673041e-05,
         7.15787974e-05, 3.21137777e-05, 2.83963673e-05, 4.43809622e-05,
         5.39421053e-05, 1.14610571e-04, 1.39578930e-04, 1.06732157e-04,
         4.26052720e-05, 2.48067554e-05, 7.06428691e-05, 3.88324443e-05,
         4.64208024e-05, 5.52886013e-05, 3.12440170e-05, 7.95435626e-05,
         4.55041954e-05, 4.18942509e-05, 3.38547252e-05, 5.18217166e-05,
         8.13050938e-05, 7.36464935e-05, 3.83299230e-05, 4.51461587e-04,
         1.45598417e-04, 6.32630181e-05

In [43]:
# Index of the neuron with the highest value/activation in the output layer
np.argmax(predictions, axis=1)

array([386], dtype=int64)

In [44]:
# Neuron number 386 had the highest activation
predictions[0][386]

0.6365591

In [45]:
# Let's get the label of the neuron with the highest activation
for name, desc, score in decode_predictions(predictions, top=5)[0]:
    print("{}: {} ({:.2%})".format(name, desc, score))

n02504458: African_elephant (63.66%)
n01871265: tusker (24.58%)
n02504013: Indian_elephant (4.18%)
n04507155: umbrella (0.09%)
n02091134: whippet (0.08%)


Looks like the classification worked! MobileNetV2 correctly classified the image as an elephant.

### Example 3: Transfer learning using an existing deep neural network

Use an already trained model that's good at a certain task and slightly modify it to do task it was not trained to do but still benefit from all the original training/learning.

We use the MobileNetV2 model again to classify an image but instead of classifying the type of animal, we want the model to identify whether the animal is male or female. Here we will use images of cats.

In [46]:
# Import modules
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import numpy as np
from imageio import imread
from skimage.transform import resize
from keras import Model
from keras.layers import Dense

In [47]:
# Load pretrained model
model = MobileNetV2(weights="imagenet")

In [48]:
# Load data
data = np.empty((40, 224, 224, 3))
# Note the order of operations matters! (i.e. first resize then preprocess won't work)
for i in range(20):
    im = imread("data/images/cat_images/f{:02d}.jpg".format(i + 1))
    im = preprocess_input(im) # preprocess the input to have pixel values between -1 and 1
    im = resize(im, output_shape=(224, 224))
    data[i] = im
for i in range(20):
    im = imread("data/images/cat_images/m{:02d}.jpg".format(i + 1))
    im = preprocess_input(im)
    im = resize(im, output_shape=(224, 224))
    data[i + 20] = im

In [49]:
# Generate labels
labels = np.empty(40, dtype=int)
labels[:20] = 0 # male
labels[20:] = 1 # female

In [50]:
# Generate predictions for cat faces as sanity check whether the model recognizes all pictures as cats
predictions = model.predict(data)
for decoded_prediction in decode_predictions(predictions, top=1):
    for name, desc, score in decoded_prediction:
        print("- {} ({:.2f}%)".format(desc, score * 100))

- Siamese_cat (13.36%)
- Angora (36.42%)
- tabby (42.23%)
- tiger_cat (69.27%)
- tabby (66.41%)
- Egyptian_cat (30.08%)
- Siamese_cat (50.26%)
- Persian_cat (73.12%)
- tabby (84.13%)
- Persian_cat (20.30%)
- tabby (54.74%)
- tabby (51.40%)
- Egyptian_cat (53.58%)
- tabby (48.33%)
- Egyptian_cat (35.86%)
- tabby (76.39%)
- tabby (14.12%)
- Egyptian_cat (11.90%)
- Egyptian_cat (43.96%)
- Egyptian_cat (52.69%)
- Egyptian_cat (42.92%)
- Egyptian_cat (51.84%)
- Egyptian_cat (45.82%)
- tabby (19.49%)
- tabby (67.26%)
- tabby (61.09%)
- Egyptian_cat (43.09%)
- Egyptian_cat (33.90%)
- lynx (32.44%)
- Egyptian_cat (40.90%)
- Egyptian_cat (22.28%)
- Egyptian_cat (30.10%)
- Egyptian_cat (36.34%)
- Siamese_cat (58.66%)
- Egyptian_cat (29.08%)
- Egyptian_cat (29.34%)
- Egyptian_cat (21.58%)
- Egyptian_cat (18.75%)
- tiger_cat (69.53%)
- tabby (43.84%)


Note: just because the score is pretty low it does not mean the model is unsure whether it is a cat or not. This just indicates that there are plenty of different cat types that could be a classification of the cat image (i.e. it knows it's a cat just not as well which type of cat).

It is working decently in identifying cats in general.

#### Modify the model

There are various ways to modify a model. However, we will take all the layers of the existing model except the last, and then take the output of the second-to-last layer and connect it to our newly created output layer.

In [51]:
type(model.summary())

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

NoneType

Inspecting the *whole* summary output shows us that second-to-last layer (global_average_pooling2d) has 1280 units/neurons and the last/output layer (predictions) has 1000.

Let's now connect the second-to-last layer with a new one.

In [52]:
cat_output = Dense(2, activation="softmax")
cat_output = cat_output(model.layers[-2].output) # output second to last layer
cat_input = model.input
# Create a new model
cat_model = Model(inputs=cat_input, outputs=cat_output)

At this point we do not want to compile and train the model because we would re-train the weights of all the already existing layers. We only want to train the newly created output layer and freeze the old layers.

In [53]:
for layer in cat_model.layers[:-1]:
    layer.trainable = False

#### Compile the model

In [54]:
cat_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

#### Train the model

In [55]:
cat_model.fit(
    x=data,
    y=labels,
    epochs=20,
    verbose=2,
)

Epoch 1/20
2/2 - 1s - loss: 0.8998 - accuracy: 0.3500 - 1s/epoch - 714ms/step
Epoch 2/20
2/2 - 0s - loss: 0.8083 - accuracy: 0.5250 - 385ms/epoch - 192ms/step
Epoch 3/20
2/2 - 0s - loss: 0.7399 - accuracy: 0.5750 - 428ms/epoch - 214ms/step
Epoch 4/20
2/2 - 0s - loss: 0.6611 - accuracy: 0.5750 - 381ms/epoch - 191ms/step
Epoch 5/20
2/2 - 0s - loss: 0.6159 - accuracy: 0.5750 - 429ms/epoch - 215ms/step
Epoch 6/20
2/2 - 0s - loss: 0.5695 - accuracy: 0.6500 - 409ms/epoch - 204ms/step
Epoch 7/20
2/2 - 0s - loss: 0.5200 - accuracy: 0.7500 - 375ms/epoch - 187ms/step
Epoch 8/20
2/2 - 0s - loss: 0.4805 - accuracy: 0.7750 - 420ms/epoch - 210ms/step
Epoch 9/20
2/2 - 0s - loss: 0.4463 - accuracy: 0.8500 - 390ms/epoch - 195ms/step
Epoch 10/20
2/2 - 0s - loss: 0.4152 - accuracy: 0.8750 - 380ms/epoch - 190ms/step
Epoch 11/20
2/2 - 0s - loss: 0.3839 - accuracy: 0.9000 - 376ms/epoch - 188ms/step
Epoch 12/20
2/2 - 0s - loss: 0.3546 - accuracy: 0.9500 - 378ms/epoch - 189ms/step
Epoch 13/20
2/2 - 0s - loss:

#### Generate predictions for the training data

In [56]:
predictions = cat_model.predict(data)
# 40 images with 2 predictions (here the output neurons)
predictions.shape

(40, 2)

In [57]:
# Which of the two output neurons is the most activated for each image?
np.argmax(predictions, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

The model seems to be pretty quickly able to perfectly classify male and female cats (accuracy=1) even though the images are extremely similarly looking (i.e. this is a very difficult task). Is it really that good or should we be suspicious?


Always ask the question of what the model is actually classifying! 

Could it be that the model actually just memorized each of the images because there were so few?

This appears to be a classic case of overfitting and thus highlights again the need for a separate training and test set to validate the predictions.

#### Generate seperate training and validation sets

In [58]:
# This is a verbose way of the splitting up our data into training and validation sets but illustrates the point better
# The first 15 images for male and female cats will be used for training
training_data = np.empty((30, 224, 224, 3))
training_data[:15] = data[:15]
training_data[15:] = data[20:35]
training_labels = np.empty(30)
training_labels[:15] = 0
training_labels[15:] = 1
# The last 5 images for male and female cats will be used for validation
validation_data = np.empty((10, 224, 224, 3))
validation_data[:5] = data[15:20]
validation_data[5:] = data[35:]
validation_labels = np.empty(10)
validation_labels[:5] = 0
validation_labels[5:] = 1

We now train our cat model again but not the already existing one because that would be cheating. So we create a new one, train and fit it.

In [59]:
cat_output2 = Dense(2, activation='softmax')
cat_output2 = cat_output2(model.layers[-2].output)
cat_input2 = model.input
cat_model2 = Model(inputs=cat_input2, outputs=cat_output2)
for layer in cat_model2.layers[:-1]:
    layer.trainable = False
cat_model2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [60]:
# Split train and test data less verbose this time
cat_model2.fit(
    x=training_data,
    y=training_labels,
    validation_data=(validation_data, validation_labels),
    epochs=20,
    verbose=2
)

Epoch 1/20
1/1 - 2s - loss: 0.8713 - accuracy: 0.4667 - val_loss: 0.7983 - val_accuracy: 0.4000 - 2s/epoch - 2s/step
Epoch 2/20
1/1 - 0s - loss: 0.7798 - accuracy: 0.4000 - val_loss: 0.7605 - val_accuracy: 0.5000 - 436ms/epoch - 436ms/step
Epoch 3/20
1/1 - 0s - loss: 0.7351 - accuracy: 0.5000 - val_loss: 0.7500 - val_accuracy: 0.5000 - 365ms/epoch - 365ms/step
Epoch 4/20
1/1 - 0s - loss: 0.6908 - accuracy: 0.5667 - val_loss: 0.7383 - val_accuracy: 0.6000 - 400ms/epoch - 400ms/step
Epoch 5/20
1/1 - 0s - loss: 0.6389 - accuracy: 0.6333 - val_loss: 0.7249 - val_accuracy: 0.5000 - 432ms/epoch - 432ms/step
Epoch 6/20
1/1 - 0s - loss: 0.5857 - accuracy: 0.6333 - val_loss: 0.7149 - val_accuracy: 0.5000 - 362ms/epoch - 362ms/step
Epoch 7/20
1/1 - 0s - loss: 0.5380 - accuracy: 0.7333 - val_loss: 0.7101 - val_accuracy: 0.5000 - 437ms/epoch - 437ms/step
Epoch 8/20
1/1 - 0s - loss: 0.4983 - accuracy: 0.8667 - val_loss: 0.7079 - val_accuracy: 0.4000 - 392ms/epoch - 392ms/step
Epoch 9/20
1/1 - 0s - 

...and indeed. We see overfitting because while the training data is perfectly classified, accuracy on the test set is around chance.

Telling apart female from male cats just based on frontal cat pictures seems to be hard especially when your data set is quite small.